In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy
from shapely.geometry import Point, LineString
import os
import nept

from loading_data import get_data

In [ ]:
import info.r067d1 as info
from run import spike_sorted_infos
infos = spike_sorted_infos

In [ ]:
thisdir = os.getcwd()
pickle_filepath = os.path.join(thisdir, "cache", "pickled")
output_filepath = os.path.join(thisdir, "plots", "maze-zones")
if not os.path.exists(output_filepath):
    os.makedirs(output_filepath)

In [ ]:
def expand_line(start_pt, stop_pt, line, expand_by):
    """Expands shapely line into a zone.

    Parameters
    ----------
    start_pt : shapely.Point
    stop_pt : shapely.Point
    line : shapely.LineString
    expand_by : int or float

    Returns
    -------
    zone : shapely.Polygon

    """
    line_expanded = line.buffer(expand_by)
    zone = start_pt.union(line_expanded).union(stop_pt)

    return zone


def find_zones(info, remove_feeder, expand_by):
    """Finds zones from ideal trajectories.

    Parameters
    ----------
    info : shortcut module
    remove_feeder: boolean
    expand_by : int or float
        Amount to expand the line.

    Returns
    -------
    zone : dict
        With shapely.Polygon as values.
        Keys are u, shortcut, novel.

    """
    u_line = LineString(info.u_trajectory)
    shortcut_line = LineString(info.shortcut_trajectory)
    novel_line = LineString(info.novel_trajectory)

    feeder1 = Point(info.path_pts['feeder1'][0], info.path_pts['feeder1'][1]).buffer(expand_by*1.2)
    feeder2 = Point(info.path_pts['feeder2'][0], info.path_pts['feeder2'][1]).buffer(expand_by*1.2)

    u_zone = expand_line(Point(info.u_trajectory[0]), 
                         Point(info.u_trajectory[-1]), 
                         u_line, expand_by)
    shortcut_zone = expand_line(Point(info.shortcut_trajectory[0]), 
                                Point(info.shortcut_trajectory[-1]), 
                                shortcut_line, expand_by)
    novel_zone = expand_line(Point(info.novel_trajectory[0]), 
                             Point(info.novel_trajectory[-1]), 
                             novel_line, expand_by)

    zone = dict()
    zone['u'] = u_zone
    zone['shortcut'] = shortcut_zone.difference(u_zone)
    zone['shortcut'] = zone['shortcut'].difference(novel_zone)
    zone['novel'] = novel_zone.difference(u_zone)

    if remove_feeder:
        for feeder in [feeder1, feeder2]:
            zone['u'] = zone['u'].difference(feeder)
            zone['shortcut'] = zone['shortcut'].difference(feeder)
            zone['novel'] = zone['novel'].difference(feeder)

    return zone

In [ ]:
for info in infos:
    print(info.session_id)
    events, position, spikes, _, _ = get_data(info)
    xedge, yedge = nept.get_xyedges(position, binsize=8)

    binned_maze_shape = (len(yedge)-1, len(xedge)-1)
    xx, yy = np.meshgrid(xedge, yedge)

    zones = find_zones(info, remove_feeder=True, expand_by=15)

    xcenters = xedge[:-1] + (xedge[1:] - xedge[:-1]) / 2
    ycenters = yedge[:-1] + (yedge[1:] - yedge[:-1]) / 2

    u_zone = np.zeros(binned_maze_shape).astype(bool)
    shortcut_zone = np.zeros(binned_maze_shape).astype(bool)
    novel_zone = np.zeros(binned_maze_shape).astype(bool)

    for i, x in enumerate(xcenters):
        for j, y in enumerate(ycenters):
            if zones["u"].contains(Point(x,y)):
                u_zone[j][i] = True
            elif zones["shortcut"].contains(Point(x,y)):
                shortcut_zone[j][i] = True
            elif zones["novel"].contains(Point(x,y)):
                novel_zone[j][i] = True

    sliced_position = position.time_slice(info.task_times["phase3"].start, info.task_times["phase3"].stop)
    occupancy = nept.get_occupancy(sliced_position, yedge, xedge)

    u_pos = np.zeros(binned_maze_shape).astype(bool)
    u_pos[occupancy > 0.] = True
    u_area = np.zeros(binned_maze_shape).astype(bool)
    u_area[u_pos & u_zone] = True

    shortcut_pos = np.zeros(binned_maze_shape).astype(bool)
    shortcut_pos[(occupancy > 0.) & (~u_area)] = True
    shortcut_area = np.zeros(binned_maze_shape).astype(bool)
    shortcut_area[shortcut_pos & shortcut_zone] = True

    novel_pos = np.zeros(binned_maze_shape).astype(bool)
    novel_pos[(occupancy > 0.) & (~u_area)] = True
    novel_area = np.zeros(binned_maze_shape).astype(bool)
    novel_area[novel_pos & novel_zone] = True

    combined = np.zeros(binned_maze_shape)
    combined[u_area] = 1
    combined[shortcut_area] = 2
    combined[novel_area] = 3

    plt.figure()
    pp = plt.pcolormesh(xx, yy, combined, cmap="PuBuGn")
    plt.plot(position.x, position.y, "k.", ms=2)
    plt.tight_layout()
    filename = info.session_id + "_maze-zones.png"
    plt.savefig(os.path.join(output_filepath, filename))
    plt.close()
#     plt.show()

In [ ]:
import info.r068d6 as info

In [ ]:
_, position, _, _, _ = get_data(info)

In [ ]:
plt.plot(position.x, position.y, "r.", ms=5)
plt.plot(53, 133, "k.")
plt.plot(53, 123, "k.")
plt.show()

In [ ]:
idx = nept.find_nearest_idx(position.time, 2574.3)

In [ ]:
idx = 26963

In [ ]:
s = 1000
e = 50000

In [ ]:
ee = s+200
# plt.plot(position.time[idx], position.y[idx], "b.", ms=20)
plt.plot(position.x[s:e], position.y[s:e], "r.", ms=4)
# plt.plot(position.time[s:ee], position.y[s:ee], "k.", ms=10)
plt.show()

In [ ]:
position.time[idx]

In [ ]:
position.time[s:e]

In [ ]:
81925+7